In [1]:
#import os
#from osgeo import gdal
import numpy as np
import math
import json
import copy
#  jr,  judging whether the grid is in the river，is = 1, not = 0
# flow_acc, the raster(converted to ASCII code) that shows the accumulated flow to each cell.


## 1 读取txt到数组

In [2]:
b = np.loadtxt('ASCII/b.txt',skiprows = 6)
p = np.loadtxt('ASCII/p.txt',skiprows = 6)
fd = np.loadtxt('ASCII/fd.txt',skiprows = 6)
jr = np.loadtxt('ASCII/jr.txt',skiprows = 6)
slope = np.loadtxt('ASCII/slope.txt',skiprows = 6)
flow_acc = np.loadtxt('ASCII/flow_acc.txt',skiprows = 6)

In [3]:
print(max(b[np.where(fd == -9999)])) # = -9999
print(min(b[np.where(fd != -9999)])) # != -9999

-9999.0
0.121975


In [4]:
print(max(p[np.where(fd == -9999)])) # = -9999
print(min(p[np.where(fd != -9999)])) # != -9999

0.001131134
0.0


In [5]:
print(max(jr[np.where(fd == -9999)])) # = -9999
print(min(jr[np.where(fd != -9999)])) # != -9999

-9999.0
0.0


In [6]:
print(max(slope[np.where(fd == -9999)])) # = -9999
print(min(slope[np.where(fd != -9999)])) # != -9999

-9999.0
0.0


In [7]:
print(max(flow_acc[np.where(fd == -9999)])) # = -9999
print(min(flow_acc[np.where(fd != -9999)])) # != -9999

-9999.0
0.0


In [16]:
flow_acc.shape 

(533, 489)

## 1.1 例子遍历数组

## 2 得到起点索引列表

In [80]:
def origin_list(flow_acc, jr):
    '''
   output: A list of all starting points for all flow paths
    '''
    list_0 = []
    [rows, cols] = flow_acc.shape 
    for i in range(rows): 
        for j in range(cols): 
            if flow_acc[i,j] == 0 and jr[i, j] == 0: #  jr[i, j] == 0 起点处于非河道
                ori = tuple([[i],[j]]) # 起点非河道的行列好转元组
                list_0.append(ori)
    return list_0

In [81]:
ori_list = origin_list(flow_acc,jr) # A list of all starting points for all flow paths

In [82]:
print(ori_list[0])
print(flow_acc[ori_list[0]])
print(jr[ori_list[0]])

([1], [174])
[0.]
[0.]


In [83]:
len(ori_list)

52521

## 3 起点路径列表

In [109]:
ori_list[0:2]

[([1], [174]), ([1], [175])]

In [151]:
def flow_path(ori, jr,fd):
    '''
   output:  lists of all path points for all flow paths
    '''
    list1 = []
    row = ori[0][0]
    col = ori[1][0]
    while jr[row,col] != 1 and fd[row,col] != -9999:
        if [row,col] not in list1:
            path_0 = tuple([[row],[col]])
            list1.append(path_0)
            if fd[row,col] == 1:
                col += 1
            elif fd[row,col] == 2:
                row += 1
                col += 1
            elif fd[row,col] == 4:
                row += 1
            elif fd[row,col] == 8:
                row += 1
                col -= 1
            elif fd[row,col] == 16:
                col -= 1
            elif fd[row,col] == 32:
                row -= 1
                col -= 1
            elif fd[row,col] == 64:
                row -= 1
            elif fd[row,col] == 128:
                row -= 1
                col += 1
    path_1 = tuple([[row],[col]])  # 水体或流域出口的行列号，元组格式
    list1.append(path_1) # 包含受内水体和流域出口
    return list1

# flow_path(ori_list[0], jr,fd)

def river_paths(ori_list, jr,fd):
    '''
   output:  lists of all path points for all flow paths
    '''
    list0 = []
    for ori in ori_list:
        list1 = flow_path(ori, jr,fd)
        if jr[list1[-1]][0] == 1:
            list0.append(list1)
    return list0
# river_paths(ori_list, jr,fd))

def bound_paths(ori_list, jr,fd):
    '''
   output:  lists of all path points for all flow paths
    '''
    list0 = []
    for ori in ori_list:
        list1 = flow_path(ori, jr,fd)
        if jr[list1[-1]][0] == -9999:
            list0.append(list1)
    return list0

In [152]:
river_s = river_paths(ori_list, jr,fd)

In [153]:
bound_s = bound_paths(ori_list, jr,fd)

In [133]:
print(len(river_s))
print(len(bound_s))

51136
1385


In [158]:
river_s[0:1]

[[([1], [174]),
  ([2], [175]),
  ([3], [176]),
  ([4], [177]),
  ([5], [178]),
  ([5], [179]),
  ([5], [180]),
  ([6], [181]),
  ([7], [182]),
  ([8], [183]),
  ([9], [183]),
  ([10], [183]),
  ([11], [183]),
  ([12], [183]),
  ([13], [183]),
  ([14], [183]),
  ([15], [182]),
  ([16], [182]),
  ([17], [182]),
  ([18], [182]),
  ([19], [183]),
  ([20], [184]),
  ([21], [185]),
  ([22], [186]),
  ([23], [186]),
  ([24], [186]),
  ([25], [187]),
  ([26], [187]),
  ([27], [187]),
  ([28], [187]),
  ([29], [186]),
  ([30], [187]),
  ([31], [187]),
  ([32], [187]),
  ([33], [187]),
  ([34], [187]),
  ([35], [186]),
  ([36], [186]),
  ([37], [186]),
  ([38], [186]),
  ([39], [187]),
  ([40], [188]),
  ([41], [189]),
  ([42], [190]),
  ([43], [190]),
  ([44], [191]),
  ([45], [191]),
  ([46], [192]),
  ([47], [193]),
  ([48], [194]),
  ([49], [195]),
  ([50], [196]),
  ([51], [197]),
  ([52], [198]),
  ([53], [199]),
  ([54], [200]),
  ([55], [201]),
  ([56], [201]),
  ([57], [202]),
  ([58],

In [159]:
bound_s[0:1]

[[([1], [175]), ([0], [176])]]

In [160]:
jr[river_s[0][-1]] # 位于河道

array([1.])

In [137]:
all_paths_0 = {'river_s': river_s, 'bound_s': bound_s}

In [3]:
def write_dict(variable,name):
    ''' input: variable to write
       inout: name, string file name
        output: a txt named name.txt
    '''
    import json
    js = json.dumps(variable)   
    file = open('ASCII/{}.txt'.format(name), 'w')  
    file.write(js)  
    file.close()  
#write_dict(all_paths_0,'all_paths_0')
def read_dict(name):
    ''' input: a txt name, filename to read
    '''
    file = open('ASCII/{}.txt'.format(name), 'r') 
    js = file.read()
    dic = json.loads(js)   
    return (dic)
    file.close() 
# file=open('ASCII/all_paths.txt','w') 
# file.write(str(all_paths)); 
# file.close() 

In [139]:
write_dict(all_paths_0,'all_paths_0')

In [4]:
all_paths = read_dict('all_paths_0')

In [5]:
river_s = all_paths['river_s']

In [6]:
bound_s = all_paths['bound_s']

In [7]:
river_s[0]

[[[1], [174]],
 [[2], [175]],
 [[3], [176]],
 [[4], [177]],
 [[5], [178]],
 [[5], [179]],
 [[5], [180]],
 [[6], [181]],
 [[7], [182]],
 [[8], [183]],
 [[9], [183]],
 [[10], [183]],
 [[11], [183]],
 [[12], [183]],
 [[13], [183]],
 [[14], [183]],
 [[15], [182]],
 [[16], [182]],
 [[17], [182]],
 [[18], [182]],
 [[19], [183]],
 [[20], [184]],
 [[21], [185]],
 [[22], [186]],
 [[23], [186]],
 [[24], [186]],
 [[25], [187]],
 [[26], [187]],
 [[27], [187]],
 [[28], [187]],
 [[29], [186]],
 [[30], [187]],
 [[31], [187]],
 [[32], [187]],
 [[33], [187]],
 [[34], [187]],
 [[35], [186]],
 [[36], [186]],
 [[37], [186]],
 [[38], [186]],
 [[39], [187]],
 [[40], [188]],
 [[41], [189]],
 [[42], [190]],
 [[43], [190]],
 [[44], [191]],
 [[45], [191]],
 [[46], [192]],
 [[47], [193]],
 [[48], [194]],
 [[49], [195]],
 [[50], [196]],
 [[51], [197]],
 [[52], [198]],
 [[53], [199]],
 [[54], [200]],
 [[55], [201]],
 [[56], [201]],
 [[57], [202]],
 [[58], [201]],
 [[59], [201]],
 [[60], [201]],
 [[61], [201]],
 [[

## 4 计算动力作用
(1) 初始化计算栅格非空值为1

(2) 计算1+坡度的3次幂

(3) 计算L

(4) 计算pl

(5) 计算最终结果

In [36]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为-9999
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            ind = tuple(j)
            arr_tif[ind] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

pow_arr = ini_tif(fd,1,jr,bound_s,-9999)
reg_arr = ini_tif(fd,0,jr,bound_s,-9999)

In [32]:
print(fd[390,488])
print(pow_arr[390,488])
print(reg_arr[512,256])

32.0
1.0
0.0


In [165]:
# river_s[0]

In [189]:
# distance_list(1, river_s[0])
# rows_cols(1, river_s[0])
# jr[[0,1],[1,2]]
# pl_slope2(pow_arr,1, river_s[0],slope)
tuple(river_s[0][0])
# reg_arr[river_s[0]]

([1], [174])

In [37]:
def distance_list(ind, path_one):
    '''input: ind,某河流路径的某索引位置
      input: path_0, 某河流路径的位置列表
      output: ind下游的所有距离
    '''
    list0 = []
    if ind < len(path_one) - 2:
        d = 0
        while ind < len(path_one) - 2:
            if abs(path_one[ind][0][0]- path_one[ind+1][0][0]) + abs(path_one[ind][1][0]- path_one[ind+1][1][0]) == 2:
                d += math.sqrt(2)
            else:
                d += 1
            list0.append(d)
            ind +=1
        return list0
    else:
        print ('ind out of index of path_one')
# distance_list(1, river_s[0])

def rows_cols(ind, path_one):
    rows = []
    cols = []
    if ind < len(path_one) - 2:
        while ind < len(path_one) - 2:
            rows += path_one[ind+1][0]
            cols += path_one[ind+1][1]
            ind +=1
        return [rows,cols]
    else:
        print ('ind out of index of path_one')
# rows_cols(1, river_s[0])
# [[3,4,150],[3,4,150]]

def pl_slope(p_arr,ind, path_one,slope):
    '''p_arr: 动力作用栅格
    '''
    row = path_one[ind][0][0]
    col = path_one[ind][1][0]
    theta = slope[row,col]
    p_i = 0.5*p_arr[row,col] # 动力应小于1
    list0 = distance_list(ind, path_one)
    list1 = [pow(p_i,i)*(1+theta**0.3) for i in list0]
    return list1
# pl_slope(pow_arr,1, river_s[0],slope)
# [1.2224826413270593,
#  1.2224826413270593,
#  1.2224826413270593,
#     ...
#     ...]
def pl_slope2(p_arr,ind, path_one,slope):
    row = path_one[ind][0][0]
    col = path_one[ind][1][0]
    theta = slope[row,col]
    p_i = 0.5*p_arr[row,col] # 动力应小于1
    list0 = distance_list(ind, path_one)
    list1 = [pow(p_i,i)*(1+0.1*theta**0.3) for i in list0]
    return list1
# pl_slope2(pow_arr,1, river_s[0],slope)


def pow_all1(pow_arr0,p_arr,slope,all_paths,reg_arr0):
    pow_arr = copy.deepcopy(pow_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(len(path)-2):
            ind_0 = tuple(path[ind])
#             print(ind_0)
            if reg_arr[ind_0][0] == 0:
                pl_s = pl_slope(p_arr,ind, path,slope)
#                 print(pl_s)
                row_col = tuple(rows_cols(ind, path)) # 转为元组
#                 print(row_col)
                pow_arr[row_col] += pl_s 
#                 print(pow_arr[row_col])
                reg_arr[ind_0] = 1
            else:
                break
    return pow_arr
#pow_result = pow_all(pow_arr,p,slope,all_paths)
def pow_all2(pow_arr0,p_arr,slope,all_paths,reg_arr0):
    pow_arr = copy.deepcopy(pow_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(len(path)-2):
            ind_0 = tuple(path[ind])
#             print(ind_0)
            if reg_arr[ind_0][0] == 0:
                pl_s = pl_slope2(p_arr,ind, path,slope)
#                 print(pl_s)
                row_col = tuple(rows_cols(ind, path)) # 转为元组
#                 print(row_col)
                pow_arr[row_col] += pl_s 
#                 print(pow_arr[row_col])
                reg_arr[ind_0] = 1
            else:
                break
    return pow_arr

In [38]:
pow_result1 = pow_all1(pow_arr,p,slope,river_s,reg_arr) # 约3-8min

In [39]:
pow_result2 = pow_all2(pow_arr,p,slope,river_s,reg_arr) # 约3-8min

In [40]:
np.savetxt('ASCII/pow_all_1.txt',pow_result1,fmt='%0.6f') # 没有前6行

In [41]:
np.savetxt('ASCII/pow_all_2.txt',pow_result2,fmt='%0.6f') # 没有前6行

In [42]:
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()

In [43]:
add_begin('pow_all_1')

In [44]:
add_begin('pow_all_2')

In [79]:
pow_result1[245,256]

1.0

In [80]:
pow_result2[282,238]

1.0

### test1 数学公式测试

In [2]:
pow(2,3)

8

In [48]:
2**3

8

In [25]:
math.sqrt(2)

1.4142135623730951

In [22]:
abs(-2)

2

### test3 array 赋值

In [131]:
arr = np.arange(63).reshape((7,9))
print(arr)
#ss = [[1,0], [5, 3]]
#print(arr[ss])
print(arr[[1, 5, 6, 2], [0, 3, 1, 2]])
#arr[[1, 5, 6, 2], [0, 3, 1, 2]] += [0,1,2,3]
#arr[[1, 5, 6, 2], [0, 3, 1, 2]] *= [0,1,2,3]
#print(arr[[1,0], [5, 3]])
print(arr[[1,0], [5, 3]].prod())
print(arr[[1, 5, 6, 2], [0, 3, 1, 2]]) # 打印arr[1, 0]、arr[5, 3]，arr[7, 1]和arr[2, 2]

[[ 0  1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16 17]
 [18 19 20 21 22 23 24 25 26]
 [27 28 29 30 31 32 33 34 35]
 [36 37 38 39 40 41 42 43 44]
 [45 46 47 48 49 50 51 52 53]
 [54 55 56 57 58 59 60 61 62]]
[ 9 48 55 20]
42
[ 9 48 55 20]


## 5 计算阻力作用

### (1) 初始化判断栅格

In [45]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为-9999
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            ind = tuple(j)
            arr_tif[ind] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

reg_arr = ini_tif(fd,0,jr,bound_s,-9999) # if value == 0,需要计算
bv_arr_1 = ini_tif(fd,1,jr,bound_s,-9999) # 乘
bv_arr = ini_tif(fd,1,jr,bound_s,-9999)

In [46]:
print(jr[390,488])
print(reg_arr[390,488])
print(bv_arr_1[390,488])
print(bv_arr[512,256])

0.0
0.0
1.0
1.0


### (2) 计算每个栅格的阻力

In [47]:
def b_theta1(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                arr_tif[i, j] = (1-b[i, j])*(1+slope[i, j]**0.3)
    return arr_tif
def b_theta2(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                if slope[i, j] != 0:
                    arr_tif[i, j] = (1-0.1*b[i, j]*(1+0.1*slope[i, j]**0.3))
                else:
                    arr_tif[i, j] = (1-0.1*b[i, j])
    return arr_tif
b_th1 = b_theta1(bv_arr_1, b, slope)
b_th2 = b_theta2(bv_arr_1, b, slope)

In [48]:
def b_theta3(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                if slope[i, j] != 0:
                    arr_tif[i, j] = (1-0.06*b[i, j]*(1+0.1*slope[i, j]**0.3))
                else:
                    arr_tif[i, j] = (1-0.06*b[i, j])
    return arr_tif
b_th3 = b_theta3(bv_arr_1, b, slope)

In [49]:
def b_theta4(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                if slope[i, j] != 0:
                    arr_tif[i, j] = (1-0.02*b[i, j]*(1+0.1*slope[i, j]**0.3))
                else:
                    arr_tif[i, j] = (1-0.02*b[i, j])
    return arr_tif
b_th4 = b_theta4(bv_arr_1, b, slope)

In [50]:
print(np.max(b_th1))
print(np.max(b_th2))
print(np.max(b_th3))
print(np.max(b_th4))
print(np.where(b_th4 == 1))

3.308136854367154
0.9878025
0.9926815
0.9975605
(array([], dtype=int64), array([], dtype=int64))


### (3) 计算阻力

In [23]:
# river_s[0]

In [24]:
# jr[river_s[0][-2]]

In [25]:
# rows_cols(0, river_s[0])

In [51]:
def rows_cols(ind, path_one):
    rows = []
    cols = []
    if ind < len(path_one) - 2:
        while ind < len(path_one) - 2:
            rows += path_one[ind+1][0]
            cols += path_one[ind+1][1]
            ind +=1
        return [rows,cols]
    else:
        print ('ind out of index of path_one')
def bv_path(all_paths,reg_arr0,bv_arr0,b_th):
    bv_arr = copy.deepcopy( bv_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for i in range(0,len(path)-2):
        ind_0 = tuple(path[i])
        print(ind_0)
        print(reg_arr[ind_0][0])
        if reg_arr[ind_0][0] ==0:
            ind_1 = tuple(rows_cols(i, path))
            print(ind_1)
            bv_arr[ind_0] = b_th[ind_1].prod()
            reg_arr[ind_0] = 1
        else:
            break
    return bv_arr
def bv_paths(all_paths,reg_arr0,bv_arr0,b_th):
    bv_arr = copy.deepcopy( bv_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths: ##################################
        for i in range(0,len(path)-2):
            ind_0 = tuple(path[i])
#             print(ind_0)
#             print(reg_arr[ind_0][0])
            if reg_arr[ind_0][0] ==0:
                ind_1 = tuple(rows_cols(i, path))
                #print(ind_1)
                bv_arr[ind_0] = b_th[ind_1].prod()
                reg_arr[ind_0] = 1
            else:
                break
    return bv_arr

In [52]:
bv_result1 = bv_paths(river_s,reg_arr,bv_arr,b_th1) #
bv_result2 = bv_paths(river_s,reg_arr,bv_arr,b_th2) #

In [53]:
bv_result3 = bv_paths(river_s,reg_arr,bv_arr,b_th3) #

In [54]:
bv_result4 = bv_paths(river_s,reg_arr,bv_arr,b_th4) #

In [81]:
print(np.max(bv_result1))
print(np.max(bv_result2))
print(np.max(bv_result3))
print(np.max(bv_result4))

3.545584576241991e+38
1.0
1.0
1.0


In [30]:
print(b_th4[150,220])
print(bv_result4[150,220])

0.9975605
1.0


In [90]:
print(b[2,183])
print(slope[2,183])
print(b_th4[2,183])
print(jr[2,183])
print(fd[2,183])
print(bv_result4[2,183])

0.577
15.17954
0.9858503432255911
0.0
4.0
1.0


0.9858503432255911

In [84]:
jr[1,182]

0.0

In [25]:
bv_result4[390, 488]

1.0

In [26]:
b_th4[390, 488]

1.0

In [27]:
np.where(b_th4 > 1)

(array([], dtype=int64), array([], dtype=int64))

In [28]:
np.where(bv_result2 > 1)

(array([], dtype=int64), array([], dtype=int64))

In [55]:
np.savetxt('ASCII/bv_result_1.txt',bv_result1,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/bv_result_2.txt',bv_result2,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/bv_result_3.txt',bv_result3,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/bv_result_4.txt',bv_result4,fmt='%0.6f') # 没有前6行

In [56]:
add_begin('bv_result_1')
add_begin('bv_result_2')
add_begin('bv_result_3')
add_begin('bv_result_4')

### test 4

In [53]:
import numpy as np

import matplotlib.pyplot as plt

x=np.linspace(-5,5,1000)  #这个表示在-5到5之间生成1000个x值

y1=[i**2 for i in x]  #对上述生成的1000个数循环用sigmoid公式求对应的y

plt.plot(x,y1)  #用上述生成的1000个xy值对生成1000个点

plt.show()  #绘制图像

## 6 动力乘阻力

In [57]:
#动力结果
pow_all_1 = np.loadtxt('ASCII/pow_all_1.txt',skiprows = 6)
pow_all_2 = np.loadtxt('ASCII/pow_all_2.txt',skiprows = 6)

In [58]:
#阻力结果
bv_result1 = np.loadtxt('ASCII/bv_result_1.txt',skiprows = 6)
bv_result2 = np.loadtxt('ASCII/bv_result_2.txt',skiprows = 6)
bv_result3 = np.loadtxt('ASCII/bv_result_3.txt',skiprows = 6)
bv_result4 = np.loadtxt('ASCII/bv_result_4.txt',skiprows = 6)

In [44]:
np.max(pow_all_1)

1365.022537

In [45]:
np.max(pow_all_2)

835.881939

In [46]:
np.max(bv_result4)

1.0

In [47]:
pow_all_1[512,256]

25.058441

In [48]:
pow_all_2[512,256]

11.505845

In [49]:
bv_result1[512,256]

433819937.998455

In [50]:
bv_result2[512,256]

0.078872

In [51]:
bv_result3[512,256]

0.219428

In [52]:
bv_result4[512,256]

0.604592

In [75]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为-9999
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            ind = tuple(j)
            arr_tif[ind] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows): 
        for j in range(cols): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

am_1 = ini_tif(fd,0,jr,bound_s,-9999)
am_2 = ini_tif(fd,0,jr,bound_s,-9999)
am_3 = ini_tif(fd,0,jr,bound_s,-9999)
am_4 = ini_tif(fd,0,jr,bound_s,-9999)

def mul(arr0,arr1,arr2,d):
    arr0 = copy.deepcopy(arr0)
    [rows, cols] = arr1.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr1[i,j] != -9999:
                if arr1[i,j] != -9999 or arr2[i,j] != -9999:
                    arr0[i,j] = arr1[i,j]*arr2[i,j]
                else:
                    arr0[i,j] = d
    return arr0

In [76]:
am1 = mul(am_1,pow_all_2,bv_result1,0)

In [77]:
am1[512,256]

436283167.60641026

In [78]:
am2 = mul(am_2,pow_all_2,bv_result2,0)

In [79]:
am2[512,256]

0.079319835216

In [80]:
am3 = mul(am_3,pow_all_2,bv_result3,0)

In [81]:
am3[512,256]

0.22067391218400004

In [82]:
am4 = mul(am_4,pow_all_2,bv_result4,0)

In [83]:
am4[512,256]

0.6080248733760001

In [84]:
np.savetxt('ASCII/am1.txt',am1,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/am2.txt',am2,fmt='%0.6f') # 没有前6行

In [85]:
np.savetxt('ASCII/am3.txt',am3,fmt='%0.6f') # 没有前6行

In [86]:
np.savetxt('ASCII/am4.txt',am4,fmt='%0.6f') # 没有前6行

In [87]:
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()

In [88]:
add_begin('am1')
add_begin('am2')

In [89]:
add_begin('am3')

In [90]:
add_begin('am4')

### test5 array

In [4]:
a1=np.array([[1,2,3],[4,5,6],[7,8,9]])

In [5]:
a1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [32]:
a1*a1

array([[ 1,  4,  9],
       [16, 25, 36],
       [49, 64, 81]])

### test6 追加文本

In [8]:
np.savetxt('ASCII/am.txt',a1,fmt='%0.6f') # 没有前6行

In [9]:
with open('ASCII/am.txt', 'r+') as f:
    content = f.read() 
    print(content)
    f.seek(0, 0)
    f.write('writer:Fatsheep\n'+content)
    f.close()

1.000000 2.000000 3.000000
4.000000 5.000000 6.000000
7.000000 8.000000 9.000000



In [53]:
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()

In [56]:
add_begin('am')